In [1]:
import os
import sys
os.chdir('../')
sys.path.append(os.path.join(os.getcwd(), "src"))

In [2]:
from dataclasses import dataclass
from pathlib import Path
from WattPredictor.utils.helpers import *
from WattPredictor.utils.exception import *
from WattPredictor.constants import *
from WattPredictor import logger

In [3]:
@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    model_path: Path
    cutoff_date: str
    input_seq_len: int
    step_size: int
    img_path: Path
    metrics_path: Path

@dataclass(frozen=True)
class FeatureStoreConfig:
    hopsworks_project_name: str
    hopsworks_api_key: str

In [4]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_PATH,
                 params_filepath = PARAMS_PATH,
                 schema_filepath = SCHEMA_PATH):
        
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.training

        create_directories([config.root_dir])
        
        model_evaluation_config =  ModelEvaluationConfig(
            root_dir=Path(config.root_dir),
            model_path=Path(config.model_path),
            cutoff_date=params.cutoff_date,
            input_seq_len= params.input_seq_len,
            step_size = params.step_size,
            img_path=Path(config.img_path),
            metrics_path=Path(config.metrics_path)
        )

        return model_evaluation_config
    
    def get_feature_store_config(self) -> FeatureStoreConfig:

        config = self.config.feature_store

        feature_store_config = FeatureStoreConfig(
                hopsworks_project_name=config.hopsworks_project_name,
                hopsworks_api_key=os.environ['hopsworks_api_key'],
        )

        return feature_store_config

In [5]:
import hopsworks
import pandas as pd
import sys
import os
from WattPredictor.config.feature_config import FeatureStoreConfig
from WattPredictor.utils.exception import CustomException
from WattPredictor import logger

class FeatureStore:
    def __init__(self, config:FeatureStoreConfig):
        try:
            self.config = config
            self.connect()
        except Exception as e:
            raise CustomException(e, sys)


    def connect(self):
        try:
            self.project = hopsworks.login(
                project=self.config.hopsworks_project_name,
                api_key_value=self.config.hopsworks_api_key
            )
            self.feature_store = self.project.get_feature_store()
            self.dataset_api = self.project.get_dataset_api()
            logger.info(f"Connected to Hopsworks Feature Store: {self.config.hopsworks_project_name}")
        except Exception as e:
            raise CustomException(e, sys)


    def create_feature_group(self, name, df, primary_key, event_time, description, online_enabled=True, version=1):
        try:
            try:
                fg = self.feature_store.get_feature_group(name=name, version=version)
                logger.info(f"Feature Group '{name}' v{version} exists. Deleting it.")
                fg.delete()
            except Exception:
                logger.info(f"Feature Group '{name}' v{version} does not exist. Will create a new one.")

            # Create a new feature group
            logger.info(f"Creating Feature Group '{name}' v{version}.")
            fg = self.feature_store.get_or_create_feature_group(
                name=name,
                version=version,
                primary_key=primary_key,
                event_time=event_time,
                description=description,
                online_enabled=online_enabled
            )

            fg.save(df)
            logger.info(f"Feature Group '{name}' v{version} created and data inserted.")

        except Exception as e:
            raise CustomException(e, sys)



    def create_feature_view(self, name: str, feature_group_name: str, features: list):
        try:
            fg = self.feature_store.get_feature_group(name=feature_group_name, version=1)
            fv = self.feature_store.get_or_create_feature_view(
                name=name,
                version=1,
                query=fg.select(features),
                description=f"Feature View for {name}"
            )
            logger.info(f"Feature View '{name}' created successfully")
        except Exception as e:
            raise CustomException(e, sys)
        

    def save_training_dataset(self, feature_view_name, version_description, output_format="csv"):
        try:
            fv = self.feature_store.get_feature_view(name=feature_view_name, version=1)
            td = fv.create_training_data(
                description=version_description,
                data_format=output_format,
                write_options={"wait_for_job": True}
            )
            logger.info(f"Training dataset created for Feature View '{feature_view_name}'.")
            return td
        except Exception as e:
            raise CustomException(e, sys)
        
    def load_latest_training_dataset(self, feature_view_name):
        try:
            fv = self.feature_store.get_feature_view(name=feature_view_name, version=1)
            return fv.training_data()
        except Exception as e:
            raise CustomException(e, sys)


    def upload_file_safely(self, local_path: str, target_name: str):

        try:
            self.dataset_api.upload(
                local_path,
                f"Resources/wattpredictor_artifacts/{target_name}",
                overwrite=True 
            )
            logger.info(f"Uploaded file to Feature Store: {target_name}")
        except Exception as e:
            raise CustomException(e, sys)


    def get_training_data(self, feature_view_name: str):
        try:
            fv = self.feature_store.get_feature_view(name=feature_view_name, version=1)
            X, y = fv.training_data()
            logger.info(f"Retrieved training data from Feature View '{feature_view_name}'")
            return X, y
        except Exception as e:
            raise CustomException(e, sys)
    
    
    def get_online_features(self, feature_view_name, key_dict: dict, version=1):
        try:
            fv = self.feature_store.get_feature_view(name=feature_view_name, version=version)
            if fv is None:
                logger.error(f"[Online Fetch] Feature View '{feature_view_name}' v{version} not found.")
                raise CustomException(f"Feature View '{feature_view_name}' v{version} is None", sys)

            expected_primary_keys = ["date_str", "sub_region_code"]
            
            key_values = [key_dict[key] for key in expected_primary_keys]
            
            try:
                result = fv.get_feature_vector(key_dict)
                logger.info(f"[Online Fetch] Fetched online features using get_feature_vector for {key_dict}: {result}")
                return result
            except Exception as vector_error:
                logger.warning(f"get_feature_vector failed: {vector_error}, trying get_serving_vector")
                
                result = fv.get_serving_vector(key_values).to_dict()
                logger.info(f"[Online Fetch] Fetched online features using get_serving_vector for {key_dict}: {result}")
                return result

        except Exception as e:
            logger.error(f"[Online Fetch] Failed to fetch online features for {feature_view_name} with key {key_dict}")
            raise CustomException(e, sys)

In [ ]:
import os
import sys
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import joblib  # Add this import that was missing
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, root_mean_squared_error
from WattPredictor.utils.helpers import create_directories, save_json
from WattPredictor.utils.exception import CustomException
from WattPredictor import logger
from WattPredictor.utils.ts_generator import features_and_target


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig, feature_store_config):
        self.config = config
        self.feature_store = FeatureStore(feature_store_config)

    def evaluate(self):
        try:
            df, _ = self.feature_store.load_latest_training_dataset("elec_wx_features_view")
            df = df[['date', 'demand', 'sub_region_code', 'temperature_2m']]
            df.sort_values("date", inplace=True)
            
            train_df, test_df = df[df['date'] < self.config.cutoff_date], df[df['date'] >= self.config.cutoff_date]

            test_x, test_y = features_and_target(test_df, input_seq_len=self.config.input_seq_len, step_size=self.config.step_size)
            test_x.drop(columns=["date"], errors="ignore", inplace=True)

            model_registry = self.feature_store.project.get_model_registry()
            model_name = "wattpredictor_lightgbm"
            
            models = model_registry.get_models(model_name)
            if not models:
                raise CustomException(f"No models found with name '{model_name}'", sys)
            
            latest_model = models[0] 
            
            
            model_dir = latest_model.download()
            model_path = os.path.join(model_dir, "model.joblib")
            model_instance = joblib.load(model_path)

            preds = model_instance.predict(test_x)

            mse = mean_squared_error(test_y, preds)
            mae = mean_absolute_error(test_y, preds)
            rmse = np.sqrt(mse)
            r2 = r2_score(test_y, preds)
            mape = np.mean(np.abs((test_y - preds) / test_y)) * 100 if np.any(test_y != 0) else np.inf
            adjusted_r2 = 1 - (1 - r2) * (len(test_y) - 1) / (len(test_y) - test_x.shape[1] - 1)

            metrics = {
                "mse": mse,
                "mae": mae,
                "rmse": rmse,
                "mape": mape,
                "r2_score": r2,
                "adjusted_r2": adjusted_r2
            }

            create_directories([os.path.dirname(self.config.metrics_path)])
            save_json(self.config.metrics_path, metrics)
            logger.info(f"Saved evaluation metrics at {self.config.metrics_path}")

            fig, ax = plt.subplots(figsize=(12, 6))
            ax.plot(test_y[:100], label="Actual", color="blue")
            ax.plot(preds[:100], label="Predicted", color="red")
            ax.set_title("Predicted vs Actual (First 100 Points)")
            ax.set_xlabel("Time Step")
            ax.set_ylabel("Electricity Demand")
            ax.legend()

            create_directories([os.path.dirname(self.config.img_path)])
            fig.savefig(self.config.img_path)
            plt.close()
            logger.info(f"Saved prediction plot at {self.config.img_path}")

            self.feature_store.upload_file_safely(self.config.metrics_path, "eval/metrics.json")
            self.feature_store.upload_file_safely(self.config.img_path, "eval/pred_vs_actual.png")

            logger.info("Evaluation results uploaded to Hopsworks dataset storage")

            return metrics

        except Exception as e:
            raise CustomException("Model evaluation failed", e)

In [7]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    feature_store_config = config.get_feature_store_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config, feature_store_config=feature_store_config)
    model_evaluation.evaluate()
except Exception as e:
    raise CustomException(str(e), sys)

[2025-07-15 16:04:58,810: INFO: helpers: yaml file: config_file\config.yaml loaded successfully]
[2025-07-15 16:04:58,810: INFO: helpers: yaml file: config_file\schema.yaml loaded successfully]
[2025-07-15 16:04:58,828: INFO: helpers: yaml file: config_file\params.yaml loaded successfully]
[2025-07-15 16:04:58,830: INFO: helpers: created directory at: artifacts]
[2025-07-15 16:04:58,832: INFO: helpers: created directory at: artifacts/model_evaluation]
[2025-07-15 16:04:58,833: INFO: external: Initializing external client]
[2025-07-15 16:04:58,833: INFO: external: Base URL: https://c.app.hopsworks.ai:443]
[2025-07-15 16:05:01,810: INFO: python: Python Engine initialized.]

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1240214
[2025-07-15 16:05:04,577: INFO: 3903490710: Connected to Hopsworks Feature Store: WattPredictor]
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.50s) 
[2025-07-15 16:05:14,541: WARNING: warnings: VersionWarni

Generating TS features: 100%|██████████| 11/11 [00:00<00:00, 22.11it/s]


Downloading: 0.000%|          | 0/1596871 elapsed<00:00 remaining<?

[2025-07-15 16:05:24,406: INFO: 3216663514: Evaluation Metrics:]
[2025-07-15 16:05:24,406: INFO: 3216663514:   mse: 4699.247044481665]
[2025-07-15 16:05:24,406: INFO: 3216663514:   mae: 39.73111615456085]
[2025-07-15 16:05:24,406: INFO: 3216663514:   rmse: 68.55105429154001]
[2025-07-15 16:05:24,406: INFO: 3216663514:   mape: 3.563778192919172]
[2025-07-15 16:05:24,406: INFO: 3216663514:   r2_score: 0.9970653667195876]
[2025-07-15 16:05:24,406: INFO: 3216663514:   adjusted_r2: 0.9945197517504782]
[2025-07-15 16:05:24,406: WARNING: 3216663514: Failed to log metrics to model registry: save() missing 1 required positional argument: 'model_path']
[2025-07-15 16:05:24,406: WARNING: 3216663514: Failed to update model description: save() missing 1 required positional argument: 'model_path']
[2025-07-15 16:05:24,406: INFO: helpers: created directory at: artifacts\model_evaluation]
[2025-07-15 16:05:24,406: INFO: helpers: json file saved at: artifacts\model_evaluation\metrics.json]
[2025-07-15 

Uploading f:\WattPredictor\artifacts\model_evaluation\metrics.json: 0.000%|          | 0/206 elapsed<00:00 rem…

[2025-07-15 16:05:27,461: INFO: 3903490710: Uploaded file to Feature Store: eval/metrics.json]


Uploading f:\WattPredictor\artifacts\model_evaluation\pred_vs_actual.png: 0.000%|          | 0/94825 elapsed<0…

[2025-07-15 16:05:31,018: INFO: 3903490710: Uploaded file to Feature Store: eval/pred_vs_actual.png]
[2025-07-15 16:05:31,018: INFO: 3216663514: Evaluation results uploaded to Hopsworks dataset storage]
